In [1]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
from random import randint, choice
from tqdm import tqdm, trange
import time

In [2]:
img = plt.imread("c2.jpg")
face = plt.imread("f2.jpg")
PATCH_SIZE = (70, 70)
OVERLAP = 25
print(img.shape)

def template_match_ssd_with_mask(img, temp):
    res = np.zeros((img.shape[0] - temp.shape[0] + 1, img.shape[1] - temp.shape[1] + 1))
    for i in range(img.shape[0] - temp.shape[0] + 1):
        for j in range(img.shape[1] - temp.shape[1] + 1):
            cost = np.sum(np.square(img[i:i+temp.shape[0], j:j+temp.shape[1]] - temp))
            res[i][j] = cost
    return res

def get_random_patch(img, PATCH_SIZE):
    i = randint(0, img.shape[0] - PATCH_SIZE[0])
    j = randint(0, img.shape[1] - PATCH_SIZE[1])
    return img[i : i + PATCH_SIZE[0], j : j + PATCH_SIZE[1]]

USE_BUILTIN_LIB = True

def get_best_random_patch(img, temp):
    if USE_BUILTIN_LIB:
        res = cv.matchTemplate(img, temp, cv.TM_SQDIFF)
    else:
        res = template_match_ssd_with_mask(img, temp)
        
    ind = choice(np.argwhere(res <= 1.02*np.min(res)))
    return img[ind[0]:ind[0]+temp.shape[0], ind[1]:ind[1]+temp.shape[1]]

(299, 450, 3)


In [3]:


img_final = face.copy()

for l in trange(0, face.shape[0], PATCH_SIZE[0] - OVERLAP):
    for k in range(0, face.shape[1], PATCH_SIZE[1] - OVERLAP):
        patch = img_final[l:l+PATCH_SIZE[0], k:k+PATCH_SIZE[1]]
        if patch.shape[0] < OVERLAP or patch.shape[1] < OVERLAP:
            continue
        if l == 0:
            if k == 0:
                res = get_random_patch(img, PATCH_SIZE)
            else:
                res = get_best_random_patch(img, patch).copy()
                over = np.square(np.sum(res[:, :OVERLAP], axis=2) - np.sum(patch[:, :OVERLAP],axis=2))
                pos_tmp = []
                pos = []
                for i in range(1, patch.shape[0]):
                    for j in range(OVERLAP):
                        if j == 0:
                            over[i][j] = over[i][j] + min(over[i - 1][j], over[i - 1][j + 1])
                        elif j == OVERLAP - 1:
                            over[i][j] = over[i][j] + min(over[i - 1][j], over[i - 1][j - 1])
                        else:
                            over[i][j] = over[i][j] + min(over[i - 1][j], min(over[i - 1][j + 1] , over[i - 1][j - 1]))
                
                pos_tmp.append(np.argmin(over[-1, :]))
                for i in range(over.shape[0] - 2, -1, -1):
                    m = pos_tmp[-1]
                    if m == 0:
                        pos_tmp.append(m + np.argmin(over[i, m:m+2]))
                    elif m == OVERLAP-1:
                        pos_tmp.append(m - 1 + np.argmin(over[i, m-1:m+1]))
                    else:
                        pos_tmp.append(m - 1 + np.argmin(over[i, m-1:m+2]))
                pos = pos_tmp[::-1]
                for i in range(0, patch.shape[0]):
                    for j in range(pos[i]):
                        res[i][j] = patch[i][j]
                
                    
        else:
            if k == 0:
                
                res = get_best_random_patch(img, patch).copy()
                over = np.square(np.sum(res[:OVERLAP, :], axis=2) - np.sum(patch[:OVERLAP, :],axis=2))
                
                pos_tmp = []
                pos = []
                for j in range(1, over.shape[1]):
                    for i in range(over.shape[0]):
                        if i == 0:
                            over[i][j] = over[i][j] + min(over[i + 1][j - 1], over[i][j - 1])
                        elif i == OVERLAP - 1:
                            over[i][j] = over[i][j] + min(over[i - 1][j - 1], over[i][j - 1])
                        else:
                            over[i][j] = over[i][j] + min(over[i + 1][j - 1], min(over[i - 1][j - 1] , over[i][j - 1]))
                
                pos_tmp.append(np.argmin(over[:, -1]))
                for j in range(over.shape[1] - 2, -1, -1):
                    m = pos_tmp[-1]
                    if m == 0:
                        pos_tmp.append(m + np.argmin(over[m:m+2, j]))
                    elif m == OVERLAP-1:
                        pos_tmp.append(m - 1 + np.argmin(over[m-1:m+1, j]))
                    else:
                        pos_tmp.append(m - 1 + np.argmin(over[m-1:m+2, j]))
                pos = pos_tmp[::-1]
                for j in range(0, patch.shape[1]):
                    for i in range(pos[j]):
                        res[i][j] = patch[i][j]
            else:
                res = get_best_random_patch(img, patch).copy()
                
                over = np.square(np.sum(res[:, :OVERLAP], axis=2) - np.sum(patch[:, :OVERLAP],axis=2))
                pos_tmp = []
                pos = []
                for i in range(1, patch.shape[0]):
                    for j in range(OVERLAP):
                        if j == 0:
                            over[i][j] = over[i][j] + min(over[i - 1][j], over[i - 1][j + 1])
                        elif j == OVERLAP - 1:
                            over[i][j] = over[i][j] + min(over[i - 1][j], over[i - 1][j - 1])
                        else:
                            over[i][j] = over[i][j] + min(over[i - 1][j], min(over[i - 1][j + 1] , over[i - 1][j - 1]))
                
                pos_tmp.append(np.argmin(over[-1, :]))
                for i in range(over.shape[0] - 2, -1, -1):
                    m = pos_tmp[-1]
                    if m == 0:
                        pos_tmp.append(m + np.argmin(over[i, m:m+2]))
                    elif m == OVERLAP-1:
                        pos_tmp.append(m - 1 + np.argmin(over[i, m-1:m+1]))
                    else:
                        pos_tmp.append(m - 1 + np.argmin(over[i, m-1:m+2]))
                pos = pos_tmp[::-1]
                for i in range(0, patch.shape[0]):
                    for j in range(pos[i]):
                        res[i][j] = patch[i][j]
                        

                over = np.square(np.sum(res[:OVERLAP, :], axis=2) - np.sum(patch[:OVERLAP, :],axis=2))
                pos_tmp = []
                pos = []
                for j in range(1, over.shape[1]):
                    for i in range(over.shape[0]):
                        if i == 0:
                            over[i][j] = over[i][j] + min(over[i + 1][j - 1], over[i][j - 1])
                        elif i == OVERLAP - 1:
                            over[i][j] = over[i][j] + min(over[i - 1][j - 1], over[i][j - 1])
                        else:
                            over[i][j] = over[i][j] + min(over[i + 1][j - 1], min(over[i - 1][j - 1] , over[i][j - 1]))
                
                pos_tmp.append(np.argmin(over[:, -1]))
                for j in range(over.shape[1] - 2, -1, -1):
                    m = pos_tmp[-1]
                    if m == 0:
                        pos_tmp.append(m + np.argmin(over[m:m+2, j]))
                    elif m == OVERLAP-1:
                        pos_tmp.append(m - 1 + np.argmin(over[m-1:m+1, j]))
                    else:
                        pos_tmp.append(m - 1 + np.argmin(over[m-1:m+2, j]))
                pos = pos_tmp[::-1]
                for j in range(0, patch.shape[1]):
                    for i in range(pos[j]):
                        res[i][j] = patch[i][j]
                
        img_final[l : l + PATCH_SIZE[0], k : k + PATCH_SIZE[1]] = res
        

plt.imshow(img_final)
plt.imsave("im4.jpg", img_final)


 25%|██▌       | 14/55 [00:34<01:34,  2.30s/it]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/nimac/.pyenv/versions/3.7.5/envs/nimacenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-cc30e2e1ce37>", line 71, in <module>
    res = get_best_random_patch(img, patch).copy()
  File "<ipython-input-2-36b8957f6666>", line 24, in get_best_random_patch
    res = cv.matchTemplate(img, temp, cv.TM_SQDIFF)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nimac/.pyenv/versions/3.7.5/envs/nimacenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2034, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nimac/.pyenv/versions/

KeyboardInterrupt: 

<div dir="rtl">
طبق روش گفته شده در کلاس،‌پچی را پیدا می کنیم که با تصویر اصلی کمترین اختلاف را دارد. برای اینکار تمپلیت را از تصویر اصلی و با کمی هم پوشانی انتخاب می کنیم و در تصویر زمینه که پنبه است به دنبال قسمت مناسب می گردیم.
از روش گفته شده در سوال ۳ استفاده نموده ایم
</div>

